# Music genre classification notebook

## Importing Libraries

In [2]:
# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

## Extracting music and features

### Dataset

We use [GTZAN genre collection](http://marsyasweb.appspot.com/download/data_sets/) dataset for classification. 
<br>
<br>
The dataset consists of 10 genres i.e
 * Blues
 * Classical
 * Country
 * Disco
 * Hiphop
 * Jazz
 * Metal
 * Pop
 * Reggae
 * Rock
 
Each genre contains 100 songs. Total dataset: 1000 songs

## Extracting the Spectrogram for every Audio

In [6]:
cmap = plt.get_cmap('inferno')
path = 'data/archive/genres/'

plt.figure(figsize=(10,10))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'{path}{g}'):
        songname = f'{path}{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(
            y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

<Figure size 720x720 with 0 Axes>

## Extracting features from Spectrogram


We will extract

* Mel-frequency cepstral coefficients (MFCC)(20 in number)
* Spectral Centroid,
* Zero Crossing Rate
* Chroma Frequencies
* Spectral Roll-off.

In [7]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

## Writing data to csv file

We write the data to a csv file 

In [11]:
file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'{path}{g}'):
        songname = f'{path}{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        rmse = librosa.feature.rms(y=y)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

# Analysing the Data in Pandas

In [12]:
data = pd.read_csv('data.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00016.au,0.304322,0.081461,1455.001672,1825.885415,3009.856800,0.061351,-233.763596,120.673508,-3.171085,...,2.537329,-1.240455,8.439245,-0.210692,-3.906118,-4.478783,-0.012436,1.293613,-2.084796,blues
1,blues.00031.au,0.253487,0.151571,1331.073970,1973.643437,2900.174130,0.042967,-221.802536,110.843071,18.620985,...,-13.037724,-12.652227,-1.821906,-7.260097,-6.660252,-14.682693,-11.719264,-11.025215,-13.387259,blues
2,blues.00034.au,0.262248,0.135939,1536.783076,2157.138100,3601.028225,0.045700,-217.211197,105.791168,15.618613,...,-10.695904,-14.224881,-5.156212,-3.471578,-6.184278,-15.325319,-6.845779,-9.828355,-13.941391,blues
3,blues.00017.au,0.302096,0.093282,1088.698166,1410.947954,2134.412094,0.048324,-245.756439,140.485916,-2.764771,...,-4.008893,1.608254,5.899378,-3.022785,-4.772989,-3.025787,-1.993938,-0.055459,-3.975350,blues
4,blues.00001.au,0.340983,0.095918,1529.835316,2038.617579,3548.820207,0.056044,-207.556793,124.006721,8.930560,...,5.376803,-2.239120,4.216963,-6.012273,0.936110,-0.716537,0.293876,-0.287431,0.531573,blues


In [13]:
data.shape

(1000, 28)

In [14]:
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)

## Encoding the Labels

In [15]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

## Scaling the Feature columns

In [16]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

## Dividing data into training and Testing set

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [18]:
len(y_train)

800

In [19]:
len(y_test)

200

In [20]:
X_train[10]

array([ 0.47953418,  1.28382877,  1.41388388,  1.9524151 ,  1.812215  ,
        0.36426616,  0.84728305, -1.1678041 ,  1.84740781, -1.16196611,
        0.82213604, -0.32375636,  1.57938855, -0.47310859,  1.49824371,
       -0.19793365,  1.21718368, -0.03269625,  1.44609211,  0.78833547,
        1.83395958,  0.98792295,  1.18323175,  0.59745316,  0.24702757,
       -0.59445535])

# Classification with Keras

## Building our Network

In [21]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [22]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [23]:
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)

Epoch 1/20
7/7 [==============================] - 0s 2ms/step - loss: 2.2290 - accuracy: 0.2000
Epoch 2/20
7/7 [==============================] - 0s 4ms/step - loss: 1.8709 - accuracy: 0.3500
Epoch 3/20
7/7 [==============================] - 0s 3ms/step - loss: 1.6426 - accuracy: 0.3837
Epoch 4/20
7/7 [==============================] - 0s 3ms/step - loss: 1.4645 - accuracy: 0.4525
Epoch 5/20
7/7 [==============================] - 0s 4ms/step - loss: 1.3152 - accuracy: 0.5750
Epoch 6/20
7/7 [==============================] - 0s 4ms/step - loss: 1.1988 - accuracy: 0.6100
Epoch 7/20
7/7 [==============================] - 0s 4ms/step - loss: 1.1027 - accuracy: 0.6488
Epoch 8/20
7/7 [==============================] - 0s 4ms/step - loss: 1.0377 - accuracy: 0.6662
Epoch 9/20
7/7 [==============================] - 0s 3ms/step - loss: 0.9758 - accuracy: 0.6837
Epoch 10/20
7/7 [==============================] - 0s 4ms/step - loss: 0.9201 - accuracy: 0.7188
Epoch 11/20
7/7 [======================

In [24]:
test_loss, test_acc = model.evaluate(X_test,y_test)

7/7 [==============================] - 0s 2ms/step - loss: 0.9815 - accuracy: 0.6450


In [25]:
print('test_acc: ',test_acc)

test_acc:  0.6449999809265137


Tes accuracy is less than training dataa accuracy. This hints at Overfitting

## Validating our approach
Let's set apart 200 samples in our training data to use as a validation set:

In [26]:
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

In [58]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=100,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)

Epoch 1/100
2/2 [==============================] - 0s 78ms/step - loss: 2.3354 - accuracy: 0.0900 - val_loss: 2.2296 - val_accuracy: 0.2150
Epoch 2/100
2/2 [==============================] - 0s 24ms/step - loss: 2.2512 - accuracy: 0.1550 - val_loss: 2.1743 - val_accuracy: 0.2600
Epoch 3/100
2/2 [==============================] - 0s 21ms/step - loss: 2.1894 - accuracy: 0.1950 - val_loss: 2.1088 - val_accuracy: 0.2900
Epoch 4/100
2/2 [==============================] - 0s 23ms/step - loss: 2.1571 - accuracy: 0.1850 - val_loss: 2.0362 - val_accuracy: 0.3050
Epoch 5/100
2/2 [==============================] - 0s 20ms/step - loss: 2.0941 - accuracy: 0.2100 - val_loss: 1.9629 - val_accuracy: 0.3350
Epoch 6/100
2/2 [==============================] - 0s 19ms/step - loss: 2.0342 - accuracy: 0.2550 - val_loss: 1.8900 - val_accuracy: 0.3750
Epoch 7/100
2/2 [==============================] - 0s 28ms/step - loss: 1.9859 - accuracy: 0.2650 - val_loss: 1.8197 - val_accuracy: 0.3850
Epoch 8/100
2/2 [===

2/2 [==============================] - 0s 18ms/step - loss: 0.7545 - accuracy: 0.7483 - val_loss: 1.1290 - val_accuracy: 0.6600
Epoch 60/100
2/2 [==============================] - 0s 17ms/step - loss: 0.7549 - accuracy: 0.7617 - val_loss: 1.1320 - val_accuracy: 0.6600
Epoch 61/100
2/2 [==============================] - 0s 14ms/step - loss: 0.7468 - accuracy: 0.7550 - val_loss: 1.1326 - val_accuracy: 0.6600
Epoch 62/100
2/2 [==============================] - 0s 14ms/step - loss: 0.7227 - accuracy: 0.7550 - val_loss: 1.1365 - val_accuracy: 0.6650
Epoch 63/100
2/2 [==============================] - 0s 16ms/step - loss: 0.7257 - accuracy: 0.7450 - val_loss: 1.1487 - val_accuracy: 0.6650
Epoch 64/100
2/2 [==============================] - 0s 17ms/step - loss: 0.7031 - accuracy: 0.7683 - val_loss: 1.1550 - val_accuracy: 0.6400
Epoch 65/100
2/2 [==============================] - 0s 22ms/step - loss: 0.6992 - accuracy: 0.7650 - val_loss: 1.1466 - val_accuracy: 0.6500
Epoch 66/100
2/2 [========

In [48]:
results

[1.1452711820602417, 0.6650000214576721]

In [59]:
model.save('cls_genre')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: cls_genre/assets
